## Managing Hallucination in LLM apps using Hybrid search experience in a RAG pipeline using SingleStore databases under its Free Shared Tier.

Code setup - Download dependencies

In [4]:
!pip install wget --quiet
!pip install openai==1.3.3 --quiet
!pip install sentence-transformers --quiet

All imports

In [5]:
import json
import os
import pandas as pd
import wget
from sentence_transformers import SentenceTransformer

In [6]:
# download embedding model

model = SentenceTransformer('flax-sentence-embeddings/all_datasets_v3_mpnet-base')

Import data from a CSV file - it holds the title, summary, and category of approximately 2000 news articles.

In [8]:
# download reviews csv file
cvs_file_path = 'https://raw.githubusercontent.com/openai/openai-cookbook/main/examples/data/AG_news_samples.csv'
file_path = 'AG_news_samples.csv'

if not os.path.exists(file_path):
    wget.download(cvs_file_path, file_path)
    print('File downloaded successfully.')
else:
    print('File already exists in the local file system.')

File already exists in the local file system.


In [9]:
df = pd.read_csv('AG_news_samples.csv')
df

,title,description,label_int,label
0,World Briefings,BRITAIN: BLAIR WARNS OF CLIMATE THREAT Prime M...,1,World
1,Nvidia Puts a Firewall on a Motherboard (PC Wo...,PC World - Upcoming chip set will include buil...,4,Sci/Tech
2,"Olympic joy in Greek, Chinese press",Newspapers in Greece reflect a mixture of exhi...,2,Sports
3,U2 Can iPod with Pictures,"SAN JOSE, Calif. -- Apple Computer (Quote, Cha...",4,Sci/Tech
4,The Dream Factory,"Any product, any shape, any size -- manufactur...",4,Sci/Tech
...,...,...,...,...
1995,You Control: iTunes puts control in OS X menu ...,MacCentral - You Software Inc. announced on Tu...,4,Sci/Tech
1996,Argentina beat Italy for place in football final,Favourites Argentina beat Italy 3-0 this morni...,2,Sports
1997,NCAA case no worry for Spurrier,Shortly after Steve Spurrier arrived at Florid...,2,Sports
1998,Secret Service Busts Cyber Gangs,The US Secret Service Thursday announced arres...,4,Sci/Tech


In [10]:
data = df.to_dict(orient='records')
data[0]

{'title': 'World Briefings',
 'description': 'BRITAIN: BLAIR WARNS OF CLIMATE THREAT Prime Minister Tony Blair urged the international community to consider global warming a dire threat and agree on a plan of action to curb the  quot;alarming quot; growth of greenhouse gases.',
 'label_int': 1,
 'label': 'World'}

In [11]:
data[1333]

{'title': 'Rhode Island Senator to Stay Republican (AP)',
 'description': "AP - Republican Sen. Lincoln Chafee, who flirted with changing political parties in the wake of President Bush's re-election victory, says he will stay in the GOP.",
 'label_int': 1,
 'label': 'World'}

Set up the SingleStoreDB database 

In [12]:
shared_tier_check = %sql show variables like 'is_shared_tier'
if not shared_tier_check or shared_tier_check[0][1] == 'OFF':
    %sql DROP DATABASE IF EXISTS news;
    %sql CREATE DATABASE news;

In [13]:
%%sql
DROP TABLE IF EXISTS news_articles;
CREATE TABLE IF NOT EXISTS news_articles (
    title TEXT,
    description TEXT,
    genre TEXT,
    embedding BLOB,
    FULLTEXT (title, description)
);

++
||
++
++

Compute embeddings for every row based on the description column

In [14]:
descriptions = [row['description'] for row in data]
all_embeddings = model.encode(descriptions)
all_embeddings.shape

(2000, 768)

In [29]:
# Merge embedding values into data rows.
for row, embedding in zip(data, all_embeddings):
    row['embedding'] = embedding

An example of one row of the combined data.

In [30]:
data[0]

{'title': 'World Briefings',
 'description': 'BRITAIN: BLAIR WARNS OF CLIMATE THREAT Prime Minister Tony Blair urged the international community to consider global warming a dire threat and agree on a plan of action to curb the  quot;alarming quot; growth of greenhouse gases.',
 'label_int': 1,
 'label': 'World',
 'embedding': array([-1.42552713e-02, -1.03357071e-02,  1.25946105e-02,  8.40715785e-03,
        -6.92264410e-03, -8.77237227e-03, -5.38323671e-02,  1.95311196e-02,
         9.50564742e-02,  1.60899572e-02,  4.72200625e-02,  2.30231155e-02,
        -6.69442937e-02,  2.82599987e-03,  2.79738400e-02, -6.46088347e-02,
         5.52451760e-02, -4.02353071e-02, -2.22880822e-02, -1.65119395e-02,
         3.61824557e-02,  3.32110142e-03,  1.18329516e-02,  7.70277716e-03,
        -4.18954827e-02, -2.76368838e-02,  3.64982933e-02,  3.69321145e-02,
         5.97776957e-02,  8.05662386e-03,  3.38091105e-02, -1.52911590e-02,
         1.38111366e-02, -4.00905032e-03,  3.15332080e-08,  2.20

In [31]:
data[1333]

{'title': 'Rhode Island Senator to Stay Republican (AP)',
 'description': "AP - Republican Sen. Lincoln Chafee, who flirted with changing political parties in the wake of President Bush's re-election victory, says he will stay in the GOP.",
 'label_int': 1,
 'label': 'World',
 'embedding': array([-5.12574315e-02,  5.09770811e-02,  2.27411874e-02, -3.61131839e-02,
        -1.83073934e-02, -3.34729217e-02, -4.54292260e-02, -9.55501106e-03,
        -2.35447544e-03, -7.64554143e-02,  1.35432659e-02,  3.43418568e-02,
        -3.55032235e-02, -6.58668904e-03, -2.80331671e-02, -8.95179436e-03,
         5.67503721e-02,  6.20207377e-03,  8.38336423e-02, -3.58631127e-02,
        -6.26204209e-03, -1.30460924e-02,  3.02522499e-02,  3.79978269e-02,
        -4.51927856e-02, -2.41312720e-02, -1.07790399e-02,  1.26847411e-02,
         5.95324263e-02,  5.69594167e-02,  3.69333848e-03,  1.35077164e-02,
         7.43618654e-03, -7.59617949e-04,  2.80009775e-08, -2.15259250e-02,
        -3.47269997e-02, -

In [35]:
%sql TRUNCATE TABLE news_articles;

import sqlalchemy as sa
from singlestoredb import create_engine


conn = create_engine().connect()

statement = sa.text('''
        INSERT INTO news_articles (
            title,
            description,
            genre,
            embedding
        )
        VALUES (
            :title,
            :description,
            :label,
            :embedding
        )
    ''')

conn.execute(statement, data)

Run semantic search and get scores

In [38]:
search_query = 'Aussie'
search_embedding = model.encode(search_query)

query_statement = sa.text('''
    SELECT
        title,
        description,
        genre,
        DOT_PRODUCT(embedding, :embedding) AS score
    FROM news_articles
    ORDER BY score DESC
    LIMIT 10
    ''')


# Execute the SQL statement.
results = pd.DataFrame(conn.execute(query_statement, dict(embedding=search_embedding)))
print(results)

                                               title  \
0  All Australians accounted for in Iraq: Downer ...   
1                  A trio of television technologies   
2          National Foods posts increased net profit   
3                   Australia's leader wins 4th term   
4                    Cricket: Aussies dominate India   
5  Woman believed to be 1st to walk around Austra...   
6                    Australia clinches series sweep   
7           US buy spurs registrar #39;s share surge   
8                           Springboks targets scrum   
9                  Man tried for UK student's murder   

                                         description     genre     score  
0  AFP - Australia has accounted for all its nati...     World  0.305584  
1  AUSTRALIANS went into a television-buying fren...  Sci/Tech  0.245194  
2  Australia #39;s biggest supplier of fresh milk...  Business  0.233651  
3  SYDNEY -- Prime Minister John Howard of Austra...     World  0.216256  
4  Austr

In [42]:
search_query = 'GDP'
search_embedding = model.encode(search_query)

query_statement = sa.text('''
    SELECT
        title,
        description,
        genre,
        DOT_PRODUCT(embedding, :embedding) AS score
    FROM news_articles
    ORDER BY score DESC
    LIMIT 10
    ''')


# Execute the SQL statement.
results = pd.DataFrame(conn.execute(query_statement, dict(embedding=search_embedding)))
print(results)

                                               title  \
0                                GDP in Q3 grows 6.3   
1             Stocks Up Despite Sluggish GDP Reading   
2                              Personal income rises   
3           China's Economic Boom Still Roaring (AP)   
4       Euro-Zone Growth Continues to Lag in 2Q (AP)   
5            Ground-Level Views of What Needs Fixing   
6  U.S. July Output Up; Factories Run Faster (Reu...   
7            US lowers growth rate in second quarter   
8           Goldman reports quarterly earnings up 36   
9                     Eurozone economy keeps growing   

                                         description     genre     score  
0  DESPITE the budget deficit, continued increase...  Business  0.490065  
1  NEW YORK - A sluggish gross domestic product r...     World  0.484387  
2  The US Commerce Department said Thursday perso...  Business  0.431883  
3  AP - China's economic boom is still roaring de...  Business  0.424738  
4  AP - 

Run hybrid search and get scores: This search finds the average of the score gotten from the semantic search and the score gotten from the key-word search and sorts the news articles by this combined score to perform an effective hybrid search.

In [43]:
hyb_query = 'Articles about Aussie captures'
hyb_embedding = model.encode(hyb_query)

# Create the SQL statement.
hyb_statement = sa.text('''
    SELECT
        title,
        description,
        genre,
        DOT_PRODUCT(embedding, :embedding) AS semantic_score,
        MATCH(title, description) AGAINST (:query) AS keyword_score,
        (semantic_score + keyword_score) / 2 AS combined_score
    FROM news_articles
    ORDER BY combined_score DESC
    LIMIT 10
    ''')

# Execute the SQL statement.
hyb_results = pd.DataFrame(conn.execute(hyb_statement, dict(embedding=hyb_embedding, query=hyb_query)))
hyb_results

,title,description,genre,semantic_score,keyword_score,combined_score
0,Aussie alive after capture in Iraq,AUSTRALIAN journalist John Martinkus is lucky ...,World,0.334077,0.124661,0.229369
1,All Australians accounted for in Iraq: Downer ...,AFP - Australia has accounted for all its nati...,World,0.445396,0.000000,0.222698
2,Cricket: Aussies dominate India,Australia tighten their grip on the third Test...,World,0.368577,0.000000,0.184289
3,Air NZ: Aussie regulator granted alliance appeal,WELLINGTON: National carrier Air New Zealand s...,Business,0.254219,0.106852,0.180536
4,Man tried for UK student's murder,The trial of a man accused of murdering York b...,World,0.350485,0.000000,0.175243
5,Ponting doesn #39;t think much of Kiwis or win...,RICKY PONTING believes the game #39;s watchers...,Sports,0.345483,0.000000,0.172742
6,Hassan Body Found in Fallujah: Australian PM,Australia #39;s prime minister says a body fou...,World,0.341777,0.000000,0.170889
7,A trio of television technologies,AUSTRALIANS went into a television-buying fren...,Sci/Tech,0.332006,0.000000,0.166003
8,Australia PM Gets Down to Work on Fourth Term ...,Reuters - Australia's conservative Prime Minis...,World,0.324336,0.000000,0.162168
9,"Police pull body of lost autistic man, 46, fro...","Canadian Press - OAKVILLE, Ont. (CP) - The bod...",World,0.322738,0.000000,0.161369


In [44]:
hyb_query = 'Articles about GDP'
hyb_embedding = model.encode(hyb_query)

# Create the SQL statement.
hyb_statement = sa.text('''
    SELECT
        title,
        description,
        genre,
        DOT_PRODUCT(embedding, :embedding) AS semantic_score,
        MATCH(title, description) AGAINST (:query) AS keyword_score,
        (semantic_score + keyword_score) / 2 AS combined_score
    FROM news_articles
    ORDER BY combined_score DESC
    LIMIT 10
    ''')

# Execute the SQL statement.
hyb_results = pd.DataFrame(conn.execute(hyb_statement, dict(embedding=hyb_embedding, query=hyb_query)))
hyb_results

,title,description,genre,semantic_score,keyword_score,combined_score
0,Stocks Up Despite Sluggish GDP Reading,NEW YORK - A sluggish gross domestic product r...,World,0.398089,0.452300,0.425194
1,GDP in Q3 grows 6.3,"DESPITE the budget deficit, continued increase...",Business,0.377384,0.223494,0.300439
2,Consumers help GDP edge up just 0.1,Slumping corporate spending and exports caused...,Business,0.340264,0.160609,0.250436
3,Delta Auditor Concerned About Airline,"Delta Air Lines (DAL.N: Quote, Profile, Resear...",Business,0.077613,0.399742,0.238678
4,Government measures slow China #39;s GDP growth,Annual economic growth in China has slowed for...,Business,0.306762,0.139683,0.223223
5,Euro-Zone Growth Continues to Lag in 2Q (AP),AP - The euro-zone economy grew by 0.5 percent...,Business,0.369067,0.000000,0.184533
6,Eurozone economy keeps growing,Official figures show the 12-nation eurozone e...,Business,0.341060,0.000000,0.170530
7,Ground-Level Views of What Needs Fixing,Just as the PhD crowd emerge with different in...,Business,0.309190,0.030088,0.169639
8,U.S. July Output Up; Factories Run Faster (Reu...,Reuters - U.S. industrial output advanced in\J...,Business,0.333499,0.000000,0.166749
9,Personal income rises,The US Commerce Department said Thursday perso...,Business,0.326598,0.000000,0.163299


Clean up

Action Required

If you created a new database in your Standard or Premium Workspace, you can drop the database by running the cell below. Note: this will not drop your database for Free Starter Workspaces. To drop a Free Starter Workspace, terminate the Workspace using the UI.

In [45]:
shared_tier_check = %sql show variables like 'is_shared_tier'
if not shared_tier_check or shared_tier_check[0][1] == 'OFF':
    %sql DROP DATABASE IF EXISTS news;